# SMA Crossover Strategy
Author: TraderPy

Youtube: https://www.youtube.com/channel/UC9xYCyyR_G3LIuJ_LlTiEVQ/featured

Website: https://traderpy.com/

## Disclaimer
Trading the financial markets imposes a risk of financial loss. TraderPy is not responsible for any financial losses that viewers suffer. Content is educational only and does not serve as financial advice. Information or material is provided ‘as is’ without any warranty. 

Past trading results do not indicate future performance. Strategies that worked in the past may not reflect the same results in the future.

In [55]:
import MetaTrader5 as mt5
import pandas as pd
import plotly.express as px
import numpy as np

In [56]:
# connect to MetaTrader5 as mt5
mt5.initialize()

True

In [57]:
# settings
symbol = 'EURUSD'
timeframe = mt5.TIMEFRAME_D1
start_pos = 0
num_bars = 1000

fast_sma_period = 10
slow_sma_period = 100

In [58]:
# Requesting historical data
bars = mt5.copy_rates_from_pos(symbol, timeframe, start_pos, num_bars)

df = pd.DataFrame(bars)[['time', 'open', 'high', 'low', 'close', 'spread']]
df['time'] = pd.to_datetime(df['time'], unit='s')

df['fast_sma'] = df['close'].rolling(fast_sma_period).mean()
df['slow_sma'] = df['close'].rolling(slow_sma_period).mean()

# finding crossovers
df['prev_fast_sma'] = df['fast_sma'].shift(1)

df.dropna(inplace=True)
df

,time,open,high,low,close,spread,fast_sma,slow_sma,prev_fast_sma
99,2020-07-20,1.14107,1.14668,1.14013,1.14472,2,1.135937,1.106639,1.134550
100,2020-07-21,1.14472,1.15388,1.14219,1.15265,2,1.138475,1.106994,1.135937
101,2020-07-22,1.15268,1.16001,1.15058,1.15695,0,1.140877,1.107430,1.138475
102,2020-07-23,1.15693,1.16258,1.15394,1.15959,2,1.143990,1.107789,1.140877
103,2020-07-24,1.15959,1.16568,1.15805,1.16549,2,1.147574,1.108153,1.143990
...,...,...,...,...,...,...,...,...,...
995,2023-12-29,1.10655,1.10835,1.10331,1.10381,16,1.100055,1.075714,1.099579
996,2024-01-02,1.10430,1.10443,1.09374,1.09404,15,1.100516,1.075708,1.100055
997,2024-01-03,1.09410,1.09644,1.08922,1.09186,15,1.100475,1.075722,1.100516
998,2024-01-04,1.09157,1.09713,1.09151,1.09468,15,1.100150,1.075767,1.100475


In [59]:
# Visualize Close Price
px.line(df, x='time', y='close')

In [60]:
def find_crossover(fast_sma, prev_fast_sma, slow_sma):
    
    if fast_sma > slow_sma and prev_fast_sma < slow_sma:
        return 'bullish crossover'
    elif fast_sma < slow_sma and prev_fast_sma > slow_sma:
        return 'bearish crossover'
    
    return None


df['crossover'] = np.vectorize(find_crossover)(df['fast_sma'], df['prev_fast_sma'], df['slow_sma'])

# df.dropna(inplace=True)
# signal = df[df['crossover'] == 'bullish crossover'].copy() # => Original
signal = df[(df['crossover'] == 'bullish crossover') | (df['crossover'] == 'bearish crossover')].copy()

signal

,time,open,high,low,close,spread,fast_sma,slow_sma,prev_fast_sma,crossover
263,2021-03-09,1.18447,1.19147,1.18346,1.18992,16,1.202320,1.203199,1.204812,bearish crossover
300,2021-04-29,1.21230,1.21490,1.21009,1.21172,20,1.206023,1.205323,1.204496,bullish crossover
338,2021-06-22,1.19159,1.19515,1.18800,1.19399,16,1.203132,1.203200,1.205451,bearish crossover
339,2021-06-23,1.19355,1.19693,1.19106,1.19247,16,1.200612,1.203090,1.203132,bearish crossover
702,2022-11-14,1.03334,1.03578,1.02704,1.03250,18,1.003800,1.002821,0.999376,bullish crossover
840,2023-05-25,1.07504,1.07557,1.07065,1.07242,10,1.080501,1.081419,1.082415,bearish crossover
857,2023-06-19,1.09359,1.09454,1.09063,1.09216,10,1.081020,1.080652,1.078920,bullish crossover
901,2023-08-18,1.08665,1.08929,1.08441,1.08712,10,1.092793,1.093049,1.094180,bearish crossover
968,2023-11-21,1.09384,1.09641,1.08990,1.09111,10,1.080956,1.079108,1.078828,bullish crossover


In [61]:
# visualize close price
fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)
    
fig.show()

In [62]:
# creating backtest and position classes

class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, sl, tp):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'open'
        
    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume if self.order_type == 'buy' \
                                                                        else (self.open_price - self.close_price) * self.volume
        self.status = 'closed'
        
    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }
        
        
class Strategy:
    def __init__(self, df, starting_balance, volume):
        self.starting_balance = starting_balance
        self.volume = volume
        self.positions = []
        self.data = df
        
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df
        
    def add_position(self, position):
        self.positions.append(position)
        
        return True
        
# logic
    def run(self):
        for i, data in self.data.iterrows():
            
            if data.crossover == 'bearish crossover':
                for position in self.positions:
                    if position.status == 'open':
                        position.close_position(data.time, data.close)
                self.add_position(Position(data.time, data.close, 'sell', self.volume, 0, 0))
            
            if data.crossover == 'bullish crossover':
                for position in self.positions:
                    if position.status == 'open':
                        position.close_position(data.time, data.close)
                self.add_position(Position(data.time, data.close, 'buy', self.volume, 0, 0))
        
        return self.get_positions_df()

In [63]:
# Lot Size:
# Standard => Volume = 100,000 => Lot = 1.00 => pip Value = 10.00 USD
# Mini Lot => Volume = 10,000 => Lot = 0.10 => pip Value = 1.00 USD
# Micro Lot => Volume = 1,000 => Lot = 0.01 => pip Value = 0.10 USD
# Nano Lot => Volume = 100 => Lot = 0.0001 => pip Value = 10 USD

starting_balance = 1000 
standard_contract = 100000
initial_volume = 10000 # pip = 1 USD
initial_lot = initial_volume / standard_contract
volume = initial_lot * standard_contract 


In [64]:
sma_crossover_strategy = Strategy(df, starting_balance, volume)
deals = sma_crossover_strategy.run()

deals

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2021-03-09,1.18992,sell,10000.0,0,0,2021-04-29,1.21172,-218.0,closed,782.0
1,2021-04-29,1.21172,buy,10000.0,0,0,2021-06-22,1.19399,-177.3,closed,604.7
2,2021-06-22,1.19399,sell,10000.0,0,0,2021-06-23,1.19247,15.2,closed,619.9
3,2021-06-23,1.19247,sell,10000.0,0,0,2022-11-14,1.03250,1599.7,closed,2219.6
4,2022-11-14,1.03250,buy,10000.0,0,0,2023-05-25,1.07242,399.2,closed,2618.8
5,2023-05-25,1.07242,sell,10000.0,0,0,2023-06-19,1.09216,-197.4,closed,2421.4
6,2023-06-19,1.09216,buy,10000.0,0,0,2023-08-18,1.08712,-50.4,closed,2371.0
7,2023-08-18,1.08712,sell,10000.0,0,0,2023-11-21,1.09111,-39.9,closed,2331.1
8,2023-11-21,1.09111,buy,10000.0,0,0,NaT,NaN,NaN,open,NaN


In [65]:
#  Get Columns Titles to check Existing Columns
titles = deals.columns

for title in titles:
 print(title)


open_datetime
open_price
order_type
volume
sl
tp
close_datetime
close_price
profit
status
pnl


In [66]:
#  Get total profit
total_profit = deals['profit'].sum()

total_profit

1331.1000000000006

In [67]:
px.line(deals, x='close_datetime', y='pnl')

In [68]:
# visualize close price
fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)
    
for i, row in deals[deals['status'] == 'closed'].iterrows():
    
    if row.profit > 0:
        fig.add_shape(type="line",
            x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
            line=dict(color="Green",width=3)
                     )
                      
    elif row.profit < 0:
        fig.add_shape(type="line",
            x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
            line=dict(color="Red",width=3)
                      )

    
fig.show()

In [69]:

    
def apply_profit_type(x):
# Finding profit type
  if x['profit'] > 0: 
      return "win"
  elif x['profit'] < 0:
      return "loss"
  else: return 'pass'

deals['profit_type'] = deals.apply(apply_profit_type, axis=1)

deals.dropna(inplace=True)

display(deals)

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,profit_type
0,2021-03-09,1.18992,sell,10000.0,0,0,2021-04-29,1.21172,-218.0,closed,782.0,loss
1,2021-04-29,1.21172,buy,10000.0,0,0,2021-06-22,1.19399,-177.3,closed,604.7,loss
2,2021-06-22,1.19399,sell,10000.0,0,0,2021-06-23,1.19247,15.2,closed,619.9,win
3,2021-06-23,1.19247,sell,10000.0,0,0,2022-11-14,1.03250,1599.7,closed,2219.6,win
4,2022-11-14,1.03250,buy,10000.0,0,0,2023-05-25,1.07242,399.2,closed,2618.8,win
5,2023-05-25,1.07242,sell,10000.0,0,0,2023-06-19,1.09216,-197.4,closed,2421.4,loss
6,2023-06-19,1.09216,buy,10000.0,0,0,2023-08-18,1.08712,-50.4,closed,2371.0,loss
7,2023-08-18,1.08712,sell,10000.0,0,0,2023-11-21,1.09111,-39.9,closed,2331.1,loss


In [70]:
# Find Initial Deposit = Initial Balance 
deposit = starting_balance
print('deposit = ', deposit)

# find total net profit
total_net_profit = round(deals['profit'].sum())
print('total net profit = ', total_net_profit)

# find gross profit
gross_profit = round(sum(deals[deals['profit_type']=='win']['profit']))
print('gross profit = ', gross_profit)
# find gross loss
gross_loss = round(sum(deals[deals['profit_type']=='loss']['profit']))
print('gross loss = ', gross_loss)

#  find profit factor
profit_factor = gross_profit / gross_loss
print('profit factor = ', profit_factor)

# find largest profit trade
largest_profit_trade = round(deals['profit'].max())
print('largest profit trade = ', largest_profit_trade)

# find largest loss trade
largest_loss_trade = round(deals['profit'].min())
print('largest loss trade = ', largest_loss_trade)

deposit =  1000
total net profit =  1331
gross profit =  2014
gross loss =  -683
profit factor =  -2.9487554904831623
largest profit trade =  1600
largest loss trade =  -218


In [71]:
out = (
    deals.groupby(deals["profit_type"].ne(deals["profit_type"].shift()).cumsum(), as_index=False)
    .agg({"profit_type": "first", "profit": "sum"})
    .rename(columns={"profit": "grand_profit"})
)
display(out)

,profit_type,grand_profit
0,loss,-395.3
1,win,2014.1
2,loss,-287.7


In [72]:
# Create Summary DataFrame
summary = pd.DataFrame(columns=['fast_sma',
                                'slow_sma',
                                
                                'deposit',  # deposit
                                'balance',  # balance
                                
                                'net_profit', # total net profit
                                
                                'gross_profit', # sum of win trades
                                'gross_loss', # sum of fail trades
                                'profit_factor', # ratio (%) => (gross profit / gross loss) => one means (profit = loss)
                                
                                'abs_drawdown', # the largest loss is lower than the initial deposit value
                                'max_drawdown', # maximal loss of the local maximum in the deposit currency and in percents of the deposit
                                'rel_drawdown', #  the maximal loss in percents of the maximum equity value and its corresponding money value
                                
                                'total_trades', # count total trades
                                'total_buy', # count total BUY trades
                                'total_sell', # count total SELL trades
                                
                                'max_win', # Largest profit trade
                                'max_fail', # Largest loss trade
                                
                                'min_lot', # initial lot
                                'max_lot', # highest lot
                                
                                'F',
                                'G'])

# balance =  9971.76
# total net profit =  -28.24
# gross profit =  5.43
# gross loss =  -33.67
# profit factor =  0.16
# largest profit trade =  5.2
# largest loss trade =  -2.2
# total trades =  65
# count SELL trades =  24
# count BUY trades =  41
# count WIN trades =  5
# count LOSS trades =  60
# ave_win_value =  1.09
# ave_loss_value =  -0.56

summary

,fast_sma,slow_sma,total_net_profit,gross_profit,gross_loss,F,G
